In [ ]:
!pip install gradio
!sudo apt install -y fluidsynth
!pip install pyfluidsynth pretty_midi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbc

In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
import pretty_midi
import tensorflow as tf

# Load model
model = tf.keras.models.load_model("music_generator_model.h5", compile=False)

# Constants
vocab_size = 128
seq_length = 20
key_order = ["pitch", "step", "duration"]

# Load a seed MIDI file and convert to notes
def midi_to_notes(midi_file):
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument = pm.instruments[0]
    notes = {"pitch": [], "start": [], "end": [], "step": [], "duration": []}
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start
    for note in sorted_notes:
        start = note.start
        end = note.end
        notes["pitch"].append(note.pitch)
        notes["start"].append(start)
        notes["end"].append(end)
        notes["step"].append(start - prev_start)
        notes["duration"].append(end - start)
        prev_start = start
    return pd.DataFrame(notes)

# Load seed notes from a MIDI file
seed_midi_path = "/content/x (11).mid"  # Update path if needed
raw_notes = midi_to_notes(seed_midi_path)

# Predict next note using model
def predict_next_note(notes, keras_model, temperature):
    assert temperature > 0
    inputs = np.expand_dims(notes, 0)
    predictions = keras_model.predict(inputs, verbose=0)
    pitch_logits = predictions['pitch']
    step = predictions["step"]
    duration = predictions["duration"]

    pitch_logits /= temperature
    pitch = tf.random.categorical(pitch_logits, num_samples=1)
    pitch = tf.squeeze(pitch, axis=-1)
    step = tf.squeeze(step, axis=-1)
    duration = tf.squeeze(duration, axis=-1)
    step = tf.maximum(0, step)
    duration = tf.maximum(0, duration)
    return int(pitch), float(step), float(duration)

# Generate music
def generate_music(temperature=1.0, num_notes=300):
    sample_notes = np.stack([raw_notes[key] for key in key_order], axis=1)
    input_notes = sample_notes[:seq_length] / np.array([vocab_size, 1, 1])

    generated_notes = []
    prev_start = 0
    for _ in range(num_notes):
        pitch, step, duration = predict_next_note(input_notes, model, temperature)
        start = prev_start + step
        end = start + duration
        input_note = (pitch, step, duration)
        generated_notes.append((*input_note, start, end))
        input_notes = np.delete(input_notes, 0, axis=0)
        input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)
        prev_start = start

    df = pd.DataFrame(generated_notes, columns=(*key_order, 'start', 'end'))

    # Convert to MIDI
    instrument = pretty_midi.Instrument(program=0)
    for _, row in df.iterrows():
        note = pretty_midi.Note(
            velocity=80,
            pitch=int(row['pitch']),
            start=float(row['start']),
            end=float(row['end'])
        )
        instrument.notes.append(note)

    midi = pretty_midi.PrettyMIDI()
    midi.instruments.append(instrument)
    midi.write("generated.mid")

    # Convert MIDI to WAV using FluidSynth
    audio = midi.fluidsynth(fs=16000)
    return (16000, audio)

# Gradio UI
interface = gr.Interface(
    fn=generate_music,
    inputs=[
        gr.Slider(0.1, 5.0, value=2.0, label="Temperature"),
        gr.Slider(100, 2000, value=300, step=100, label="Number of Notes")
    ],
    outputs=gr.Audio(label="Generated Audio"),
    title="🎼 LSTM Music Generator",
    description="Play with temperature and sequence length to generate different tunes!"
)

interface.launch(share=True, debug=True)


/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fe852c1d1016f8d1ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv


/usr/local/lib/python3.11/dist-packages/gradio/processing_utils.py:749: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
